# 驗證文件檔輸出至影像

說明:
利用YOLOv4驗證測試集輸出偵測文件檔(如附件1)，將偵測文件檔結合在原始影像輸出有畫框的影像。

參數:
1. `ROOT`              : 原始影片/座標文件檔路徑
2. `SPLIT_ROOT`        : 要分割的路徑(偵測文件檔的路徑為絕對路徑 -> 分割成相對路徑顯示在輸出的影像)
3. `image_save_folder` : 儲存影片路徑
4. `color_file`        : 標籤/顏色清單文件檔 (可有可無)
5. `isAll`             : `isAll=True`:所有標籤畫出來, `isAll=False`:僅將`onlyLabels`內的標籤畫出來
6. `onlyLabels`        : 只要畫特定的標籤 (僅`isAll=False`時定義)

資料夾樹狀圖:
```
ROOT  
├─ images_out     # 程式建立  
│  ├─ 000000.jpg  # 程式輸出  
│  ├─ 000001.jpg  # 程式輸出  
│  └─ ..  
├─ comp4_det_test_cone.txt    # 將darknet/result/comp4_det_test_cone.txt複製過來  
├─ comp4_det_test_person.txt  # 將darknet/result/comp4_det_test_person.txt複製過來  
└─ class_colors.txt           # 標籤顏色清單 (可有可無)  
```

---
流程:  
1. 修改`scr/detector.c`內function:`validate_detector`約第799行前後，並重新編譯生成`detector.exe`:

    修改前:
    ```
    else {
        print_detector_detections(fps, id, dets, nboxes, classes, w, h);
    }
    ```
    
    修改後:
    ```
    else {
        print_detector_detections(fps, path, dets, nboxes, classes, w, h);
    }
    ```

2. 修改`train.data`內`valid`指到測試集路徑文件檔

3. 執行YOLOv4驗證影像輸出偵測文件檔指令(in Linux)如:  
    `./darknet detector valid ./train.data ./yolov4-tiny.cfg ./best.weights`
    => 執行上述指令`darknet/result`會有文件檔開頭為`comp4_det_test_`的`.txt`檔

4. 複製`darknet/result/comp4_det_test_*.txt`至`ROOT`底下

5. 確保原始圖片存在，原始圖片的路徑為`comp4_det_test_*.txt`內第一欄的資訊

6. 執行此`Yolov4ValidTxt2DetectImage.py`

---
附件1:

comp4_det_test_cone.txt:
```
D:\Dataset(Test)\Sumitomo(CM088A)\220316\file_20220316083442\20220208_REFERENCE_DATA_01\220208142415\220208142415_img\000013.jpg 0.001128 857.669189 270.190430 1012.778687 510.487732
D:\Dataset(Test)\Sumitomo(CM088A)\220316\file_20220316083442\20220208_REFERENCE_DATA_01\220208142415\220208142415_img\000016.jpg 0.001183 696.023315 167.945938 797.465942 397.454895
D:\Dataset(Test)\Sumitomo(CM088A)\220316\file_20220316083442\20220208_REFERENCE_DATA_01\220208142415\220208142415_img\000017.jpg 0.001583 645.638672 151.927246 746.952881 357.833893
D:\Dataset(Test)\Sumitomo(CM088A)\220316\file_20220316083442\20220208_REFERENCE_DATA_01\220208142415\220208142415_img\000022.jpg 0.001301 607.242493 104.822128 673.907166 247.199509
```

comp4_det_test_person.txt:
```
D:\Dataset(Test)\Sumitomo(CM088A)\220316\file_20220316083442\20220208_REFERENCE_DATA_01\220208142415\220208142415_img\000003.jpg 0.002355 735.588013 8.579786 796.169678 69.028137
D:\Dataset(Test)\Sumitomo(CM088A)\220316\file_20220316083442\20220208_REFERENCE_DATA_01\220208142415\220208142415_img\000004.jpg 0.002856 735.764648 7.264225 794.745361 68.298630
D:\Dataset(Test)\Sumitomo(CM088A)\220316\file_20220316083442\20220208_REFERENCE_DATA_01\220208142415\220208142415_img\000010.jpg 0.995857 1002.637329 420.485291 1220.185913 604.217163
D:\Dataset(Test)\Sumitomo(CM088A)\220316\file_20220316083442\20220208_REFERENCE_DATA_01\220208142415\220208142415_img\000011.jpg 0.998493 974.717773 354.667572 1151.623779 562.967651
```

In [1]:
import os
import cv2
import random
from tqdm.notebook import tqdm # jupyter notebook的進度條

In [2]:
# GLOBAL VARIABLE
ROOT = 'C:\\Users\\danielwu\\Desktop\\valid2image'
SPLIT_ROOT = 'D:\\Dataset(Test)\\Sumitomo(CM088A)\\'
image_save_folder = 'images_out'
color_file = 'class_colors.txt'
isAll = True
if not isAll:
    onlyLabels = ['person']

In [3]:
## @brief Description: 建立資料夾
#  @param folder
#  
#  @return None
#  @date 20220321  danielwu
def MakeDirs(folder):
    if not os.path.isdir(folder):
        os.makedirs(folder)

In [4]:
## @brief Description: 建立標籤對應的顏色
#  @param [in] name          標籤名稱
#  
#  @return 'label:(B, G, R)'
#  @date 20220406  danielwu
def ClassColors(name):
    return '{}:\t{}'.format(name, (
        random.randint(0, 255),
        random.randint(0, 255),
        random.randint(0, 255)))

In [5]:
## @brief Description: 讀取/建立標籤顏色清單
#  @param [in] color_path        標籤顏色清單文件檔
#  @param [in] labels_list       標籤清單
#  @param [in] dict_class_colors {標籤:顏色}
#  
#  @return label_list      標籤清單
#          all_detections  所有偵測訊息
#  @date 20220406  danielwu
def CheckColors(color_path, labels_list, dict_class_colors={}):
    if os.path.isfile(color_path):
        with open(color_path, 'r') as f:
            line = f.readline()
            while line:
                if '#' in line:             # ignore comments
                    line = f.readline()
                    continue
                line = line.split('\n')[0]  # remove '\n' in each line
                if line == '':              # ignore empty line
                    line = f.readline()
                    continue
                dict_class_colors[line.split(':\t')[0]] = eval(line.split(':\t')[1])  # eval() : str to tuple
                line = f.readline()
        for label in labels_list:
            if label not in dict_class_colors:
                with open(color_path, 'a') as f:
                    f.write('\n{}'.format(ClassColors(label)))
                CheckColors(color_path, labels_list, dict_class_colors)
    else:
        with open(color_path, 'w') as f:
            f.write('## ' + '='*21 + '\n')
            f.write('# "label:" + "\\t" + "(B, G, R)"\n')
            f.write('# ' + '='*22 + '\n')
            for label in labels_list:
                f.write('\n{}'.format(ClassColors(label)))
        CheckColors(color_path, labels_list, dict_class_colors)
            
    return dict_class_colors

In [6]:
## @brief Description: 取得母資料夾內所有特定格式檔案路徑
#  @param [in] folder     母資料夾路徑
#  @param [in] extension  副檔名
#  
#  @return paths
#  @date 20220321  danielwu
def GetPaths(folder, extension='.MOV'):
    paths = []

    for response in os.walk(folder):       # response = (dirpath, dirname, filenames)
        if response[2] != []:              # look for filenames
            for f in response[2]:
                if f.endswith(extension):  # only append .MOV file in paths 
                    paths.append(os.path.join(response[0], f))
    
    return paths

In [7]:
def Txt2Detections(txt_path):
    info={}
    label = txt_path.split('comp4_det_test_')[1].split('.txt')[0]
    with open(txt_path, 'r') as f:
        for line in f.readlines():
            line = line.split('\n')[0]
            if line != '':
                path, conf, left, top, right, bottom = line.split(' ')
                
                detection = (float(conf),
                             round(float(left)), round(float(top)),
                             round(float(right)), round(float(bottom)))
                
                # info = {path: detial}
                # detial = {label_1: [(detection_1), (detection_2)], label_2: [(detection_3), (detection_4)]}
                if path not in info:
                    # path not in info
                    detial = {}                      # set detial to empty
                    detections = []                  # set detection to empty
                    detections.append(detection)     # add new detection in detections
                    detial[label] = detections       # update detial
                    info[path] = detial              # update info
                else:
                    # path in info:
                    detial = info[path]               # grab original detial
                    if label not in detial:
                        # label not in detial:
                        detections = []               # set detections to empty
                        detections.append(detection)  # add new detection in detections
                        detial[label] = detections    # update detial
                        info[path] = detial           # update info
                    else:
                        # label in detial:
                        detections = detial[label]    # grab original detections
                        detections.append(detection)  # add new detection in detections
                        detial[label] = detections    # update detial
                        info[path] = detial           # update info
                    
    return info

In [8]:
def Draw(image, label, detection):
    conf, left, top, right, bottom = detection
    text_length = (len(label)+5)*12
    
    conf = conf * 100
    pos1 = (left, top)
    pos2 = (right, bottom)
    pos3 = [left - 1, top + 1]
    pos4 = [left + text_length, top - 18]
    pos5 = [left + 3, top - 3]
    
    if (left + text_length) > width:
        pos3[0] = right - text_length
        pos4[0] = right + 1
        pos5[0] = right + 3 - text_length

    if top < 18:
        pos3[1] = top -1
        pos4[1] = top + 18
        pos5[1] = top + 15
    
    color = class_colors[label]
    text = '{} {:.2f}%'.format(label, conf)
    font = cv2.FONT_HERSHEY_DUPLEX
    cv2.rectangle(image, pos1, pos2, color, 2)
    cv2.rectangle(image, pos3, pos4, color, -1)
    cv2.putText(image, text, pos5, font, .5, (0, 0, 0), 1, cv2.LINE_AA)

In [9]:
if __name__ == '__main__':
    valides_txt = GetPaths(ROOT, extension='.txt')
    valides_txt = [path for path in valides_txt if 'comp4_det_test_' in path]  # remove class_colors.txt
    labels_list = [path.split('comp4_det_test_')[1].split('.txt')[0] for path in valides_txt]
    
    color_path = os.path.join(ROOT, color_file)
    class_colors = CheckColors(color_path, labels_list)
    
    for valid_txt in tqdm(valides_txt, desc='01 valid txt'):
        detections_info = Txt2Detections(valid_txt)
        
        for path in tqdm(detections_info, desc='02 %s path' %valid_txt):
            save_path = os.path.join(ROOT, path.split(SPLIT_ROOT)[1])
            MakeDirs(os.path.split(save_path)[0])

            detial = detections_info[path]
            labels = detial.keys()
            if not os.path.isfile(save_path):
                # read original image to draw the detections for the first time
                image = cv2.imread(path)
            else:
                # read drew image whitch is the first time and draw the detections for second time
                image = cv2.imread(save_path)
            height, width = image.shape[:2]

            for label in labels:
                detections = detial[label]
                for detection in detections:
                    conf, _, _, _, _ = detection
                    if conf > 0.25:  # confidence threshold
                        if isAll:
                            # draw all detections
                            Draw(image, label, detection)
                        else:
                            # draw specific detections
                            if label in onlyLabels:
                                Draw(image, label, detection)

            # put image path into image
            text = path.split(SPLIT_ROOT)[1]
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.rectangle(image, (0, height-24), ((len(text)+10)*9, height), (255, 255, 255), -1)
            cv2.putText(image, text, (4, height-8), font, .5, (0, 0, 0), 1, cv2.LINE_AA)

            cv2.imwrite(save_path, image)
            
    print('done')

01 valid txt:   0%|          | 0/2 [00:00<?, ?it/s]

02 C:\Users\danielwu\Desktop\valid2image\comp4_det_test_cone.txt path:   0%|          | 0/2322 [00:00<?, ?it/s…

02 C:\Users\danielwu\Desktop\valid2image\comp4_det_test_person.txt path:   0%|          | 0/3885 [00:00<?, ?it…